In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "/workspaces/mirrorverse/sqlite-jdbc-3.34.0.jar")
    .config(
        "spark.driver.extraClassPath",
        "/workspaces/mirrorverse/sqlite-jdbc-3.34.0.jar")
    .getOrCreate())

ps_conn = "jdbc:sqlite:/workspaces/mirrorverse/mirrorverse.db"

In [1]:
from mirrorverse.models.chinook_depth.spark import build_training_data

depth_classes = [25, 50, 75, 100, 150, 200, 250, 300, 400, 500]
features = ["depth_class", "month", "daytime", "period_progress", "elevation"]
train_dir = "train"
test_dir = "test"

In [ ]:
build_training_data(
    spark,
    ps_conn,
    depth_classes=depth_classes,
    features=features,
    train_dir=train_dir,
    test_dir=test_dir,
    split=0.8,
    overwrite=True,
)

In [ ]:
from mirrorverse.models.chinook_depth.keras import load_data, build_model
from tensorflow.keras.layers import Dense

N = len(depth_classes)
train = load_data(train_dir, N, features, batch_size=1000, shuffle_buffer_size=10000)
test = load_data(test_dir, N, features, batch_size=1000, shuffle_buffer_size=10000)

layers = [
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu')
]

model, layers = build_model(N, features, layers)
model.fit(train, validation_data=test, epochs=10)

In [3]:
model.save("chinook_depth_model.keras")

In [4]:
from tensorflow.keras.models import load_model
new_model = load_model("chinook_depth_model.keras")

## Notes

```bash
apt-get update
apt install default-jre

curl -O https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.34.0/sqlite-jdbc-3.34.0.jar


Needed conda to get h5py installed correctly

mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-aarch64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh

~/miniconda3/bin/conda init bash
```